# Import library

In [ ]:
import copy

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Load data

In [ ]:
df = pd.read_csv('../data/data.csv')

# Data overview

In [ ]:
def describle_data():
    print('Shape of data:\n', df.shape)
    print('--'*20)
    print('Columns of data: \n', df.columns)
    print('--'*20)
    print('Data types of data: \n', df.dtypes)
    print('--'*20)
    print('Missing values of data: \n', df.isna().sum())
    print('--'*20)
    print('Describe data: \n', df.describe())
    print('--'*20)
    print('Info data: \n', df.info())
describle_data()

# Xóa bỏ những xe bị trùng lặp

In [ ]:
df.drop_duplicates(subset=['url'], inplace=True)
describle_data()

# Chuyển đổi kiểu dữ liệu cho các cột year, price sang kiểu số thực để có thể thực hiện các phép tính toán

Year

In [ ]:
def process_year_colum(year):
    if year.isdigit():
        return float(year)
    else:
        return 0
df['year'] = df['year'].astype(str).apply(process_year_colum)
describle_data()

Price

In [ ]:
df.isna().sum()
df.dropna(subset=['car_name'], inplace=True)

In [ ]:
def process_price(price):
    try:
        if price.find('Tỷ') != -1:
            ty= price.split('Tỷ')[0]
            trieu = price.split('Tỷ')[1]
            trieu = trieu.split('Triệu')[0]
            return float(ty)*1000000000 + float(trieu)*1000000
        elif price.find('Triệu') != -1:
            trieu = price.split('Triệu')[0]
            trieu = trieu.replace(' ','')
            return float(trieu)*1000000
        else:
            return 0
    except:
        return 0
df['price'] = df['price'].astype(str).apply(process_price)

In [ ]:
df['driven kms'].describe()
# lấy ra các xe có số km đi được nhỏ hơn 1000km
df[df['driven kms'] < 1000]

In [ ]:
df['price'].describe()

# Xử lý các cột dạng nhãn

Car_name có hai phần là hãng xe và tên xe, ta sẽ tách ra thành hai cột là brand và model


In [ ]:
df['brand'] = df['car_name'].str.split().str[0]
df['model'] = df['car_name'].str.split().str[1:].str.join(' ')

In [ ]:
df.head()

# Xử lý các dữ liệu bị thiếu hoặc sai


Xử lý cột year

In [ ]:
car_before_1990 = df[df['year'] < 1990]
print('Number of car before 1990: ', car_before_1990.shape[0])
print('Number of car before 1990: ', car_before_1990['year'].value_counts())

Điền các giá trị sai trong cột year bằng giá trị xuất hiện nhiều nhất trong cột year

In [ ]:
df['year'] = df['year'].apply(lambda x: df['year'].value_counts().index[0] if x < 1990 else x)

Cột price: các giá trị bằng = 0 sẽ được thay thế bằng giá trung bình của các xe sản xuất trong năm đó

In [ ]:
df['price'] = df['price'].apply(lambda x: df[df['year'] == x]['price'].mean() if x == 0 else x)
df['price'].describe()

Cột num_of_door
Thông thường các xe có tối đa 5 - 6 cửa, với những giá trị sai ta sẽ điền bằng giá trị xuất hiện nhiều nhất trong cột num_of_door

In [ ]:
df['num_of_door'] = df['num_of_door'].apply(lambda x: df['num_of_door'].value_counts().index[0] if x > 6 else x)

Cột brand

In [ ]:
df['brand'].value_counts()

Cột brand có một số hãng có số lượng xe rất ít hoặc bị thiếu, ta sẽ gộp các hãng xe này thành một nhóm là 'other'
Nếu số xe của hãng đó nhỏ hơn 10 thì sẽ gộp vào nhóm 'Other'

In [ ]:
threshold = 10
car_count = df['brand'].value_counts()
small_brands = car_count[car_count < threshold].index.tolist()
df['brand'] = df['brand'].apply(lambda x: 'Other' if x in small_brands else x)
df['brand'].fillna('Other', inplace=True)

Cột series: ta thấy các giá trị của cột series có giá trị nan, ta sẽ điền các giá trị này bằng giá trị xuất hiện nhiều nhất trong cột series

In [ ]:
df['series'] = df['series'].apply(lambda x: df['series'].value_counts().index[0] if x == '-' else x)
df['series'].fillna(df['series'].value_counts().index[0], inplace=True)

In [ ]:
df['series'].isna().sum()

Cột engine_type: ta thấy có một vài giá trị bị thiếu, ta sẽ điền các giá trị này bằng giá trị xuất hiện nhiều nhất trong cột engine_type

In [ ]:
df['engine_type'] = df['engine_type'].apply(lambda x: df['engine_type'].value_counts().index[0] if x == '-' else x)
df['engine_type'].fillna(df['engine_type'].value_counts().index[0], inplace=True)

In [ ]:
df['engine_type'].isna().sum()

Cột transmission tương tự cột engine_type

In [ ]:
df['transmission'] = df['transmission'].apply(lambda x: df['transmission'].value_counts().index[0] if x == '-' else x)
df['transmission'].fillna(df['transmission'].value_counts().index[0], inplace=True)

In [ ]:
df['transmission'].isna().sum()

# Trực quan hóa dữ liệu

Tìm hiểu về mối liên hệ giữa giá xe và các thuộc tính khác

In [ ]:
#vẽ price theo brand, series,year
plt.figure(figsize=(20, 10))
sns.barplot(x='brand', y='price', data=df)
plt.title('Price by brand')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='series', y='price', data=df)
plt.title('Price by series')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='year', y='price', data=df)
plt.title('Price by year')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='engine_type', y='price', data=df)
plt.title('Price by engine_type')
plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
sns.barplot(x='transmission', y='price', data=df)
plt.title('Price by transmission')
plt.xticks(rotation=90)
plt.subplot(1, 2, 2)
sns.barplot(x='assemble_place', y='price', data=df)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
sns.barplot(x='num_of_door', y='price', data=df)
plt.title('Price by num_of_door')
plt.subplot(1, 2, 2)
sns.barplot(x='num_of_seat', y='price', data=df)
plt.title('Price by num_of_seat')
plt.show()

# Tìm hiểu về mối liên hệ giữa các thuộc tính với nhau

In [ ]:
#vẽ biểu đồ nhiệt giữa price và các thuộc tính trừ model
plt.figure(figsize=(20, 10))
numeric_cols = df.select_dtypes(include=['float64', 'int64'])
sns.heatmap(numeric_cols.corr(), annot=True)
plt.show()




# Mã hóa các cột dạng nhãn thành dạng số để có thể sử dụng cho mô hình học máy

Cột assemble_place chỉ có 2 giá trị là 'Lắp ráp trong nước' và 'Nhập khẩu', ta sẽ mã hóa thành 0 và 1

In [ ]:
df['assemble_place'] = df['assemble_place'].map({'Lắp ráp trong nước': 0, 'Nhập khẩu': 1})

In [ ]:
df['assemble_place'].value_counts()

Cột brand sẽ được mã hóa bằng phương pháp One-hot encoding

In [ ]:
one_hot = OneHotEncoder()
one_hot.fit(df[['brand']])
one_hot_brand = pd.get_dummies(df['brand'], prefix='brand')
df = pd.concat([df, one_hot_brand], axis=1)
df.drop('brand', axis=1, inplace=True)
df.head()

Cột series sẽ được mã hóa bằng phương pháp One-hot encoding

In [ ]:
one_hot = OneHotEncoder()
one_hot.fit(df[['series']])
one_hot_series = pd.get_dummies(df['series'], prefix='series')
df = pd.concat([df, one_hot_series], axis=1)

Cột engine_type sẽ được mã hóa bằng phương pháp One-hot encoding

In [ ]:
df.head()

In [ ]:
one_hot = OneHotEncoder()
one_hot.fit(df[['engine_type']])
one_hot_engine_type = pd.get_dummies(df['engine_type'], prefix='engine_type')
df = pd.concat([df, one_hot_engine_type], axis=1)

Cột transmission chỉ có hai giá trị nên sẽ được mã hóa thành 0 và 1

In [ ]:
df['transmission'] = df['transmission'].map({'Số tay': 0, 'Số tự động': 1})

In [ ]:
df.head()

In [ ]:
save_df = copy.deepcopy(df)

In [ ]:
save_df.head()

# Sử dụng Standardization để chuẩn hóa dữ liệu

In [ ]:
scaler = StandardScaler()
df=save_df.copy()
#fit year và driven kms
scaler.fit(df[['driven kms','num_of_seat']])
df[['driven kms','num_of_seat']] = scaler.transform(df[['driven kms','num_of_seat']])

In [ ]:
df.head()

# Sử dụng hồi quy tuyến tính để dự đoán giá xe

In [ ]:
df.head()

In [ ]:
df.drop(['model', 'series', 'engine_type','car_name','url'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
model = LinearRegression()

In [ ]:
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
def describle_data():
    print('X_train shape: ', X_train.shape)
    print('X_test shape: ', X_test.shape)
    print('y_train shape: ', y_train.shape)
    print('y_test shape: ', y_test.shape)
describle_data()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
predict = model.predict(X_test)
plt.scatter(y_test, predict)
plt.xlabel('y_test')
plt.ylabel('predict')
plt.show()

In [ ]:
sns.distplot((y_test-predict),bins=50);


In [ ]:
import numpy as np
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predict))
print('MSE:', metrics.mean_squared_error(y_test, predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predict)))


In [ ]:
# thử sử dụng random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.get_params()


In [ ]:
from sklearn.model_selection import KFold, RandomizedSearchCV

"""
+ Search over space of hyperparameters to find acceptable ones
+ Two search methods implemented by scikit-learn: GridSearch and RandomSearch
+ RandomSearch is faster in case we search over large space (too many combinations of hyperparameters)
"""

rf_hyperparams = {
    'n_estimators': [200, 300, 400, 500],
    'max_depth': [None, 10, 50, 90, 110],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_randomized_search = RandomizedSearchCV(
    estimator = rf,
    param_distributions = rf_hyperparams,
    n_iter = 10,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    verbose=2,
    random_state=42,
    n_jobs = -1
)

rf_randomized_search.fit(X_train, y_train)

D:\2022.2\Old-car-price-prediction\venv\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [ ]:
rf_randomized_search.fit(X_train, y_train)


In [ ]:
rf_randomized_search.score(X_test, y_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, rf_randomized_search.predict(X_test)))
print('MSE:', metrics.mean_squared_error(y_test, rf_randomized_search.predict(X_test)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rf_randomized_search.predict(X_test))))


In [ ]:
# hồi quy ridge

In [ ]:
from sklearn.linear_model import Ridge
rg = Ridge(alpha=1.0)
rg.fit(X_train, y_train)
rg.score(X_test, y_test)


In [ ]:
#Support Vector Regression

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf')
svr.fit(X_train, y_train)
svr.score(X_test, y_test)